# Tutorial 2: `RDP` Templating

In this tutorial, we'll explore some of RadPrompter's more advanced prompting features, including:

- The `[PROMPTS]` section for defining reusable prompt components
- The `rdp` operator for composing prompts
- Using `stop_tags` to control generation

## Prompt

As before, we start by importing the `Prompt` class and loading our TOML file:

In [1]:
with open("./02_RDP-Templating.toml", "r") as f:
    lines = f.readlines()
print("".join(lines))

[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]

system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured.

Here is the report:
<report>
{{report}}
</report>"""

user_prompt_cot = """
I want you to extract the following data element from the report:
'Pulmonary Embolism'
Here are your options and you can explicitly use one of these:
- `Present`
- `Absent`

Hint: "Indicate `Present` if the report explicitly mentions the patient has pulmonary embolism in their CT scan.
Indicate `Absent` if pulmonary embolism is not seen or if a previously observed pulmonary embolism is mentioned as resolved.

After you provide the data element, I will ask you to provide an explanation and then the final answer.

Now give your initial answer. Then

This TOML file introduces a new section, `[PROMPTS]`, which allows us to define reusable prompt components. In this example, we've defined `system_prompt`, `user_prompt_intro`, and `user_prompt_cot`.

The `[CONSTRUCTOR]` section then uses these components to build the final prompt:

```toml
[CONSTRUCTOR]
system = "rdp(system_prompt)"
user = "rdp(user_prompt_intro + user_prompt_cot)"
stop_tags = "</answer>"
```

The `rdp` operator is used to reference the prompt components. If a plain string is provided instead of `rdp(...)`, that string will be used directly, just like the previous tutorial. But when `rdp` is used, RadPrompter will look up the referenced components in the `[PROMPTS]` section and concatenate them together.

This allows for a lot of flexibility in composing complex prompts from simpler pieces.

The `stop_tags` parameter is used to control generation. When the model outputs the specified tag, generation will halt. This is useful for ensuring the model follows a specific format and doesn't generate irrelevant text.

Let's create our prompt object:

In [2]:
from radprompter import Prompt

prompt = Prompt("./02_RDP-Templating.toml")
prompt

Notice `</answer>` tag after the model's `[... response ...]` generation. 

## Client & Engine

We'll use the `vLLMClient` and `RadPrompter` engine as in previous tutorials:

In [3]:
from radprompter import RadPrompter, vLLMClient

client = vLLMClient(
    model = "meta-llama/Meta-Llama-3-8B-Instruct",
    base_url = "http://localhost:9999/v1",
    temperature = 0.0,
    seed=42
)

engine = RadPrompter(
    client=client,
    prompt=prompt, 
    output_file="output_tutorial_2.csv",
    concurrency=2,
)

And we run it on our sample reports:

In [4]:
import glob

report_files = glob.glob("../../sample_reports/*.txt")

reports = []
for file in report_files:
    with open(file, "r") as f:
        reports.append({"report": f.read(), "file_name": file})

engine(reports)

Processing items: 100%|██████████| 3/3 [00:06<00:00,  2.11s/it]


The engine will process each report using our advanced prompt and save the results to `output_tutorial_2.csv`.

In [5]:
import pandas as pd

df = pd.read_csv("output_tutorial_2.csv", index_col='index')
df

,default_response,report,file_name
index,,,
1,<answer>\n<initial_answer>\nPresent\n</initial...,Here is an example radiology report describing...,../../sample_reports/sample_report_3.txt
0,<answer>\n<initial_answer>\nPresent\n</initial...,Clinical Information:\n72-year-old female with...,../../sample_reports/sample_report_2.txt
2,<answer>\n<initial_answer>\nPresent\n</initial...,Clinical Information:\n67-year-old male with s...,../../sample_reports/sample_report_1.txt


**Note**: The `stop_tag` is not returned in the LLM response. If you want that in your csv, you have to include it as a post-hoc measure:

```python

df['default_response'] = df['default_response'].apply(lambda x: x+"</answer>")

```


Finally, we save the log:

In [6]:
engine.save_log("log_tutorial_2.log")

with open("log_tutorial_2.log", "r") as f:
    print(f.read())

RadPrompter Version: 1.0.8
Model: meta-llama/Meta-Llama-3-8B-Instruct
Prompt TOML: /Users/bardiakhosravi/Desktop/GitHub/RadPrompter/tutorials/02_RDP-Templating/02_RDP-Templating.toml
Prompt Version: 0.1
Prompt Hash: b5b6e7cc73163ad0c3024020aa06a0a7
Concurrency Factor: 2
Start Time: 2024-05-19 14:36:49
End Time: 2024-05-19 14:36:55
Duration: 6.0
Number of Items: 3
Average Processing Time: 2.0


-------------------- *** - Prompt Content - *** --------------------
[METADATA]

version = 0.1
description = "A sample prompt for RadPrompter"

[PROMPTS]

system_prompt = "You are a helpful assistant that has 20 years of experience in reading radiology reports and extracting data elements."

user_prompt_intro = """
Carefully review the provided chest CT report (in the <report> tag). Ensure that each data element is accurately captured.

Here is the report:
<report>
{{report}}
</report>"""

user_prompt_cot = """
I want you to extract the following data element from the report:
'Pulmonary Embolism'

In this tutorial, we've seen how RadPrompter's `[PROMPTS]` section and `rdp` operator allow us to compose complex prompts from simpler components, and how `stop_tags` can be used to control generation. These features provide a lot of power and flexibility in designing prompts for specific tasks.